<a href="https://colab.research.google.com/github/bkodzo/ML_Project/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.0/634.0 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.4 MB/s eta 0:00:00
2.19.0-dev20250108


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-01-08 17:28:38--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.5’

train-data.tsv.5    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2025-01-08 17:28:38 (9.85 MB/s) - ‘train-data.tsv.5’ saved [358233/358233]

--2025-01-08 17:28:38--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.5’

valid-data.tsv.5    100%[========

In [ ]:
# Load the data into pandas DataFrames
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])
# Display the first few rows of the training data
print("Training Data:")
print(train_df.head())
print("\nTest Data:")
print(test_df.head())

# print column names before preprocessing
print("Train DataFrame columns before preprocessing:", train_df.columns)
print("Test DataFrame columns before preprocessing:", test_df.columns)
# %%



Training Data:
  label                                            message
0   ham  ahhhh...just woken up!had a bad dream about u ...
1   ham                           you can never do nothing
2   ham  now u sound like manky scouse boy steve,like! ...
3   ham  mum say we wan to go then go... then she can s...
4   ham  never y lei... i v lazy... got wat? dat day ü ...

Test Data:
  label                                            message
0   ham  i am in hospital da. . i will return home in e...
1   ham         not much, just some textin'. how bout you?
2   ham  i probably won't eat at all today. i think i'm...
3   ham  don‘t give a flying monkeys wot they think and...
4   ham                                who are you seeing?
Train DataFrame columns before preprocessing: Index(['label', 'message'], dtype='object')
Test DataFrame columns before preprocessing: Index(['label', 'message'], dtype='object')


In [ ]:
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')


def preprocess_text(text):
    # Remove non-alphabetic characters and punctuation, and convert to lower case
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    # Tokenize the text
    words = text.split()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Join the words back into a string
    return " ".join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Apply preprocessing to the messages
train_df['message'] = train_df['message'].apply(preprocess_text)
test_df['message'] = test_df['message'].apply(preprocess_text)





In [ ]:
# Display the first few rows of the preprocessed training data
print("Preprocessed Training Data:")
print(train_df.head())

print("\nPreprocessed Test Data:")
print(test_df.head())

Preprocessed Training Data:
  label                                            message
0   ham  ahhhhjust woken uphad bad dream u thoso dont l...
1   ham                                      never nothing
2   ham  u sound like manky scouse boy stevelike travel...
3   ham  mum say wan go go shun bian watch da glass exh...
4   ham  never lei v lazy got wat dat day send da url c...

Preprocessed Test Data:
  label                                            message
0   ham                    hospital da return home evening
1   ham                                   much textin bout
2   ham  probably wont eat today think im gonna pop wee...
3   ham  dont give flying monkeys wot think certainly d...
4   ham                                             seeing


In [ ]:
# print column names after preprocessing
print("Train DataFrame columns after preprocessing:", train_df.columns)
print("Test DataFrame columns after preprocessing:", test_df.columns)

Train DataFrame columns after preprocessing: Index(['label', 'message'], dtype='object')
Test DataFrame columns after preprocessing: Index(['label', 'message'], dtype='object')


In [ ]:
# print the types of the labels columns
print("Train DataFrame label column type after preprocessing:", train_df['label'].dtype)
print("Test DataFrame label column type after preprocessing:", test_df['label'].dtype)


Train DataFrame label column type after preprocessing: object
Test DataFrame label column type after preprocessing: object


In [ ]:
# %%
messages = pd.concat([train_df, test_df], ignore_index=True) # combine train and test data for this step
spam_messages = messages[messages["label"] == "spam"]["message"]
ham_messages = messages[messages["label"] == "ham"]["message"]
print(f"Number of spam messages: {len(spam_messages)}")
print(f"Number of ham messages: {len(ham_messages)}")

Number of spam messages: 747
Number of ham messages: 4824


In [ ]:
from collections import Counter

# Function to get top n most frequent words
def get_top_n_words(messages, n=10):
    all_words = " ".join(messages).split()
    word_counts = Counter(all_words)
    return word_counts.most_common(n)

# Get top 10 words for spam and ham
top_10_spam_words = get_top_n_words(spam_messages, 10)
top_10_ham_words = get_top_n_words(ham_messages, 10)



In [ ]:
print("\nTop 10 most frequent words in spam messages:")
print(top_10_spam_words)

print("\nTop 10 most frequent words in ham messages:")
print(top_10_ham_words)


Top 10 most frequent words in spam messages:
[('call', 350), ('free', 219), ('u', 155), ('txt', 151), ('ur', 144), ('mobile', 124), ('text', 121), ('stop', 118), ('claim', 113), ('reply', 101)]

Top 10 most frequent words in ham messages:
[('u', 997), ('im', 463), ('get', 307), ('dont', 276), ('ltgt', 276), ('ok', 273), ('go', 252), ('ur', 243), ('ill', 242), ('know', 232)]


In [ ]:
# Prepare data for the model
# Convert text to sequences of tokens
tokenizer = keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="<unk>")
tokenizer.fit_on_texts(train_df['message'])

train_sequences = tokenizer.texts_to_sequences(train_df['message'])
test_sequences = tokenizer.texts_to_sequences(test_df['message'])

# Pad sequences to have the same length
max_length = 50  # You can adjust this
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Convert labels to numpy arrays
train_labels = np.array(train_df['label'].map({'ham': 0, 'spam': 1})) # map string labels to numerical
test_labels = np.array(test_df['label'].map({'ham': 0, 'spam': 1})) # map string labels to numerical

In [ ]:
# Define the model
model = keras.Sequential([
    keras.layers.Embedding(input_dim=1000, output_dim=16, input_length=max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.build(input_shape=(None, max_length))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ (None, 50, 16)              │          16,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_6           │ (None, 16)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,289 (63.63 KB)

 Trainable params: 16,289 (63.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8411 - loss: 0.4366 - val_accuracy: 0.8657 - val_loss: 0.3575
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8680 - loss: 0.3500 - val_accuracy: 0.8635 - val_loss: 0.3418
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8672 - loss: 0.3324 - val_accuracy: 0.8628 - val_loss: 0.3113
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8600 - loss: 0.3020 - val_accuracy: 0.8606 - val_loss: 0.2592
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8771 - loss: 0.2335 - val_accuracy: 0.9102 - val_loss: 0.2082
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9182 - loss: 0.1933 - val_accuracy: 0.9555 - val_loss: 0.1784
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9478 - loss: 0.1482 - val_accuracy: 0.9648 - val_loss: 0.1157
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9715 - loss: 0.1033 - val_accuracy: 0.

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Convert text to sequence
    pred_sequence = tokenizer.texts_to_sequences([pred_text])
    # Pad the sequence
    pred_padded = keras.preprocessing.sequence.pad_sequences(pred_sequence, maxlen=max_length, padding='post', truncating='post')
    # Make prediction
    prediction_value = model.predict(pred_padded)[0][0]
    # Determine the label (ham or spam)
    if prediction_value >= 0.5:
        prediction_label = "spam"
    else:
        prediction_label = "ham"
        prediction_value = 1 - prediction_value # invert the probability for ham
    return [prediction_value, prediction_label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
[0.9561855308711529, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
You passed the challenge. Great job!
